<a href="https://colab.research.google.com/github/manalibhoir22/manali/blob/master/Product_Recommendation_2_(1)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gzip
import json
import re
import bs4
import openpyxl
import lxml
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import gzip

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def parse(path):
        g = gzip.open(path, 'rb')
        for l in g:
            yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('meta_Cell_Phones_and_Accessories.json.gz')
df

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Cell Phones & Accessories, Accessories, Scree...",,[],,Trollies Sing a Long Party VHS,[],[],,,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,,,,6303195164
1,"[Cell Phones & Accessories, Accessories, Car A...",,[],,La Pura VHS,[],[],,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,,,,630524412X
2,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[New items to give you a new good mood, welcom...",,Leopard Thanksgiving Easter PC Black Case for ...,[],[],,,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,,,,6310200240
3,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[A brand-new, unused, unopened, undamaged item...",,Puppies Faceplate Hard Case Protector for Net1...,[],[],,Live2Pedal,[Protect your phone with style through this At...,"[>#6,368,825 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,6786501710
4,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[A brand-new, unused, unopened, undamaged item...",,White Wolf Faceplate Protector Hard Case for S...,[],[],,wirelessoutletusa,[Protect your phone with style through this At...,"[>#2,672,454 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,6786501273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590066,"[Cell Phones & Accessories, Accessories, Repla...",,[Compatible with:Motorola Moto G3 XT1552 G(201...,,Full LCD Touch Screen Digitizer Assembly Repla...,[],[https://images-na.ssl-images-amazon.com/image...,,hongkongruly,[Compatible with:Motorola Moto G3 XT1552 G(201...,"[>#1,179,531 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,B01HJH76J4
590067,"[Cell Phones & Accessories, Cases, Holsters & ...",,[],,Universal Sports Armband Casehigh Shop Easy Fi...,[B01MDPWI5V],[https://images-na.ssl-images-amazon.com/image...,,CaseHQ,"[Universal Designed: Up to 5.7"" diagonal size....","663,487 in Cell Phones & Accessories (",[],{'ASIN: ': 'B01HJGI3VK'},Cell Phones & Accessories,,,$0.01,B01HJGI3VK
590068,"[Cell Phones & Accessories, Cases, Holsters & ...",,[<b>Brand Aim of LoHi:</b> <br /> LoHi is dedi...,,"iPhone 6 Plus Case, LoHi iPhone 6s Plus Case [...",[],[https://images-na.ssl-images-amazon.com/image...,,LoHi,[UNIQUE DESIGNED for Apple iPhone 6/6s Plus 5....,"[>#323,452 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,,B01HJH9IN6
590069,"[Cell Phones & Accessories, Cases, Holsters & ...",,[],,Universal Sports Armband Casehigh Shop Easy Fi...,[],[https://images-na.ssl-images-amazon.com/image...,,CaseHQ,"[Universal Designed: Up to 5.7"" diagonal size....","1,662,046 in Cell Phones & Accessories (",[B00SXRXUFE],"{'ASIN: ': 'B01HJFFF1W', 'Item model number:':...",Cell Phones & Accessories,"class=""a-bordered a-horizontal-stripes a-spa...",,$3.99,B01HJFFF1W


In [ ]:
df.shape

(590071, 18)

In [ ]:
products = pd.read_excel("data.xlsx")
products.head()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,1,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,2,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,3,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,4,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,5,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
product_ratings=pd.merge(products,df,on='asin',how='inner')
product_ratings.head()


,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,55,A2G38P52ZF5TTH,7532385086,Allana,"[3, 3]",I love this case! It's so pretty.And I love th...,5,I recommend,1296604800,"02 2, 2011","[Cell Phones & Accessories, Cases, Holsters & ...",,"[Product Description\nHTC EVO 4G, Rubberized P...",,Rubberized Purple Wave Flower Snap on Design C...,[],[],,Generic,[Rubberized Purple Wave Flower Snap on Design ...,"[>#553,803 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,
1,56,A3Q6C95EFP4FEX,7532385086,Brittany,"[0, 0]",The idea of the design is a sweet idea but it ...,3,Nice,1328140800,"02 2, 2012","[Cell Phones & Accessories, Cases, Holsters & ...",,"[Product Description\nHTC EVO 4G, Rubberized P...",,Rubberized Purple Wave Flower Snap on Design C...,[],[],,Generic,[Rubberized Purple Wave Flower Snap on Design ...,"[>#553,803 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,
2,57,A2GHDKZBYXGAPY,7532385086,"C. Groves ""~The Connie""","[0, 0]",I was worried about this order because the pic...,5,Perfect!,1342569600,"07 18, 2012","[Cell Phones & Accessories, Cases, Holsters & ...",,"[Product Description\nHTC EVO 4G, Rubberized P...",,Rubberized Purple Wave Flower Snap on Design C...,[],[],,Generic,[Rubberized Purple Wave Flower Snap on Design ...,"[>#553,803 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,
3,58,A2Y33PMFKII0VR,7532385086,Grandma in NH,"[0, 0]",Another pretty phone case that I really love. ...,5,Pretty,1360886400,"02 15, 2013","[Cell Phones & Accessories, Cases, Holsters & ...",,"[Product Description\nHTC EVO 4G, Rubberized P...",,Rubberized Purple Wave Flower Snap on Design C...,[],[],,Generic,[Rubberized Purple Wave Flower Snap on Design ...,"[>#553,803 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,
4,59,A3S8BPAUQQLDKU,7532385086,Krystal,"[0, 0]","The case is not white, its more of a silver. I...",4,Not what I expected.,1354924800,"12 8, 2012","[Cell Phones & Accessories, Cases, Holsters & ...",,"[Product Description\nHTC EVO 4G, Rubberized P...",,Rubberized Purple Wave Flower Snap on Design C...,[],[],,Generic,[Rubberized Purple Wave Flower Snap on Design ...,"[>#553,803 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,


In [ ]:
product_ratings.shape

(87746, 27)

In [ ]:
##total reviews
total=len(product_ratings)
print("Number of review:",total)
print()


## How many unique products?
print("Number of unique products:",len(product_ratings.asin.unique()))
asin_prop = float(len(product_ratings.asin.unique())/total)
print("Prop of unique products:",round(asin_prop,3))



Number of review: 87746

Number of unique products: 5109
Prop of unique products: 0.058


In [ ]:
counts=product_ratings['asin'].value_counts()
(counts>50).value_counts()

False    4830
True      279
Name: asin, dtype: int64

In [ ]:
(counts[counts>=50].index)

Index(['B005SUHPO6', 'B0090YGJ4I', 'B00BT7RAPG', 'B005SUHRVC', 'B008LFTCAK',
       'B00ANALDWQ', 'B0059DLLJC', 'B00974L0EA', 'B001XXUOQI', 'B004G1L52Q',
       ...
       'B00B78MT44', 'B00AANQLRI', 'B006WU4CQK', 'B006EEIBWU', 'B005G22I9S',
       'B0093IHX7I', 'B004LJ8N78', 'B009WDWBH6', 'B005STVU2K', 'B0054EQQXC'],
      dtype='object', length=290)

In [ ]:

product_ratings.drop(['reviewerName','helpful','reviewText','summary','unixReviewTime','reviewTime','tech1','description','fit','also_buy','image','tech2','brand','feature','rank','also_view','details','main_cat','similar_item','date','price'], axis=1, inplace=True)


In [ ]:
product_ratings.drop(product_ratings.columns[[0]],axis=1,inplace=True)

In [ ]:
product_ratings.head()

,reviewerID,asin,overall,category,title
0,A2G38P52ZF5TTH,7532385086,5,"[Cell Phones & Accessories, Cases, Holsters & ...",Rubberized Purple Wave Flower Snap on Design C...
1,A3Q6C95EFP4FEX,7532385086,3,"[Cell Phones & Accessories, Cases, Holsters & ...",Rubberized Purple Wave Flower Snap on Design C...
2,A2GHDKZBYXGAPY,7532385086,5,"[Cell Phones & Accessories, Cases, Holsters & ...",Rubberized Purple Wave Flower Snap on Design C...
3,A2Y33PMFKII0VR,7532385086,5,"[Cell Phones & Accessories, Cases, Holsters & ...",Rubberized Purple Wave Flower Snap on Design C...
4,A3S8BPAUQQLDKU,7532385086,4,"[Cell Phones & Accessories, Cases, Holsters & ...",Rubberized Purple Wave Flower Snap on Design C...


In [ ]:
df=product_ratings

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
len(df['asin'].unique())

5109

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87746 entries, 0 to 87745
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewerID  87746 non-null  object
 1   asin        87746 non-null  object
 2   overall     87746 non-null  int64 
 3   category    87746 non-null  object
 4   title       87746 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.0+ MB


In [ ]:
counts1=df['asin'].value_counts() 
counts=df['reviewerID'].value_counts()
counts1


B005SUHPO6    837
B0090YGJ4I    448
B00BT7RAPG    431
B005SUHRVC    421
B008LFTCAK    416
             ... 
B000VVDEJ2      5
B0050OV01O      5
B00BLSUUMS      5
B003MVYV2C      5
B0082C24F2      5
Name: asin, Length: 5109, dtype: int64

In [ ]:
counts

A36K2N527TXXJN    61
A1EVV74UQYVKRY    61
A1ODOGXEYECQQ8    59
A2NOW4U7W3F7RI    57
A22CW0ZHY3NJH8    56
                  ..
A2UD5EG63WWTUR     1
A2THWT0MVX5LZR     1
A1YORC7ZXY0K33     1
A3L8JOQRY8I27L     1
AEDN1IUU0RRBJ      1
Name: reviewerID, Length: 26691, dtype: int64

In [ ]:
len(counts1[counts1>=50].index)

290

In [ ]:
len(counts[counts>=50].index)

6

In [ ]:
df1=df[df['asin'].isin(counts1[counts1 >=50].index)]
df1.shape

(29167, 5)

In [ ]:
df2=df1[df1['reviewerID'].isin(counts[counts >=50].index)]
df2.shape

(57, 5)

In [ ]:
df1.head()

,reviewerID,asin,overall,category,title
379,A3CW0ZLUO5X2B1,B0009B0IX4,5,[],Plantronics Voyager 510 Bluetooth Headset [Ret...
380,ASVZC9DBNE92O,B0009B0IX4,5,[],Plantronics Voyager 510 Bluetooth Headset [Ret...
381,A3PLIXRZ6HWVTQ,B0009B0IX4,5,[],Plantronics Voyager 510 Bluetooth Headset [Ret...
382,A19RW25Q6206I3,B0009B0IX4,5,[],Plantronics Voyager 510 Bluetooth Headset [Ret...
383,AUG0C084W05KI,B0009B0IX4,4,[],Plantronics Voyager 510 Bluetooth Headset [Ret...


In [ ]:
df2.head()

,reviewerID,asin,overall,category,title
2027,A36K2N527TXXJN,B0013G8PTS,5,"[Cell Phones & Accessories, Accessories, Batte...",New Trent: iTorch 5200mAh Ultra Portable USB P...
3265,A1ODOGXEYECQQ8,B001FVPU9O,5,"[Cell Phones & Accessories, Cases, Holsters & ...","OtterBox Defender Case for iPhone 3G, 3GS (Bla..."
17123,A1ODOGXEYECQQ8,B004XZHY34,4,"[Cell Phones & Accessories, Accessories, Car A...",Jabra Freeway Bluetooth In-Car Speakerphone (U...
23012,A2NOW4U7W3F7RI,B005GGBYJ4,5,"[Cell Phones & Accessories, Accessories, Cables]",T &amp; S Electronics Micro USB OTG Cable - Mi...
24459,A22CW0ZHY3NJH8,B005LFXBJG,4,"[Cell Phones & Accessories, Accessories]",Motorola SJ1446A USB Wall Charger with Micro U...


In [ ]:
ratings_sum = pd.DataFrame(df1.groupby(['asin'])['overall'].sum()).rename(columns = {'overall': 'overall_sum'})
top10 = ratings_sum.sort_values('overall_sum', ascending = False).head(10)
top10

,overall_sum
asin,
B005SUHPO6,3656
B0090YGJ4I,1962
B00BT7RAPG,1889
B005SUHRVC,1745
B008LFTCAK,1736
B00ANALDWQ,1548
B00974L0EA,1447
B0059DLLJC,1396
B0013G8PTS,1391


In [ ]:
top10_popular=top10.merge(df1,left_index = True, right_on = 'asin').drop_duplicates(
    ['asin', 'title'])[['asin', 'title','overall_sum']]

In [ ]:
print ('Top 10 Popular Products by sum user ratings\n')
top10_popular

Top 10 Popular Products by sum user ratings



,asin,title,overall_sum
26890,B005SUHPO6,Otterbox Defender Series Hybrid Case &amp; Hol...,3656
46409,B0090YGJ4I,OtterBox Defender Series Case for iPhone 5 (Di...,1962
66980,B00BT7RAPG,Tech Armor Samsung Samsung Galaxy S4 (Not S4 A...,1889
27780,B005SUHRVC,OtterBox Commuter Series Case for iPhone 4/4S&...,1745
44474,B008LFTCAK,Generic Clear Screen Protector Shield for the ...,1736
61238,B00ANALDWQ,"iPhone 5s Case, Spigen Slim Armor Case for iPh...",1548
49928,B00974L0EA,OtterBox COMMUTER SERIES Case for iPhone 5/5s/...,1447
20241,B0059DLLJC,Snap-on Rubber Coated Case for Apple&reg; iPho...,1396
1934,B0013G8PTS,New Trent: iTorch 5200mAh Ultra Portable USB P...,1391
12640,B004G1L52Q,Mountek nGroove Universal CD Slot Mount for Ce...,1380


In [ ]:
ratingsd=df1.pivot(index='asin',columns= 'reviewerID',values='overall').fillna(0)
ratingsd.head()

reviewerID,A01038432MVI9JXYTTK5T,A01335502AS0LP3ASV1Z7,A01410041ZE038XXVXLBB,A01534622D57NTXSH2WBR,A01566963M68G4TMRL8YP,A025531519P5C6VNP05FT,A0260105268IXGY2H11N2,A027168223K80PQU4IP35,A02914441KN9I9HCGIYGD,A0388397363MZHRU6ALSX,A0464351OZXPUPKGI6HO,A04896451D6BHVAUTD75W,A05470662LAYR9LN1QIKW,A059899119L7CHCKBYQRD,A0611551KU79JNSC7JJV,A06170512LLXAS5ND7UQB,A0712184E2Z0T1Z4V4ZN,A0718784TOWF9DX1DOFD,A07961442OVXEW0GVNUET,A08699911833RNLMSSUKC,A09254782VJKWIDD9QUZM,A0954586PW98I9TNZXJP,A1000SVLSFZ8S6,A10063PJ5C9WQQ,A101577718CATXNFEYBQR,A10169B1F7EQ9T,A1018D502E27E6,A101UMMRGQJJ7X,A10204DEPQA929,A10214432MOS74F92C9QW,A10276293VA68AYD25VIZ,A102R6G6WVWF23,A102XBFOWJT4K,A103CDLPIN70O9,A103CVD599R8QA,A103D3GYGFHS96,A103EFN0LE0PPT,A1047EDJ84IMAS,A1047HEENF4C0C,A104G4NLBF3HY3,...,AZLE2XFH3JUNG,AZLFA2L5KHF3N,AZLWQV9SEZ1SS,AZMMFZKJB8PV6,AZMOM619PTO5B,AZMY6E8B52L2T,AZNQF5GS3X6CR,AZNUNL2GIX0AP,AZO17Q5R3XLYA,AZOFNVDHKYKNL,AZOJOJH5AF4TE,AZOK5STV85FBJ,AZORJBW9TZUWO,AZOW9BV7LC3SP,AZPZROZJIKBE8,AZQ8PGVNCKXJ,AZQGJ5CEAJGXB,AZQWH5DUZQIZ7,AZQYPTZWZVI18,AZR0H4PD8U0CG,AZRAGVPSIS3MN,AZRDBRM4H6333,AZSLYXFYF05R,AZT5LAAL450FN,AZTF4394JEP2Y,AZUBZF1FK4C0F,AZUR8PYQ9LFFE,AZV9TQDZU7306,AZVEABJEV1TQN,AZW6WE7UXAMU0,AZW9ENHHISX6A,AZWQEM8GKXQ5Y,AZWVLJM3958OD,AZXQ9HFT8OQZ9,AZXXVMQWISPH6,AZYA6NBTF2843,AZYJTD9J82V5I,AZZ1QIEWIUC1G,AZZVYHVJ8BDUZ,AZZYW4YOE1B6E
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
B0009B0IX4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000GAO9T2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000I8ACMU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000IEB00W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0013G8PTS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratingsd.shape

(290, 17234)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
traind, testd = train_test_split(ratingsd, test_size=0.30,random_state=42)

In [ ]:
train = traind.to_numpy()
test = testd.to_numpy()

In [ ]:
sparsity = float(len(train.nonzero()[0]))
sparsity /= (train.shape[0] * train.shape[1])
sparsity *= 100
print ('Sparsity: {:5.2f}%'.format(sparsity))

Sparsity:  0.59%


In [ ]:
def item_similarity(ratings, epsilon=1e-9):
    # epsilon -> for handling dived-by-zero errors
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [ ]:

item_sim = item_similarity(train)

In [ ]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [ ]:
item_prediction = predict_item(train, item_sim)

In [ ]:
item_prediction[:4, :4]

array([[1.04978498e-02, 1.38526865e-01, 7.64447423e-03, 1.04631164e-02],
       [2.69156938e-03, 1.13553339e-10, 1.95998545e-03, 3.82641594e-11],
       [1.67390036e-02, 4.44130306e-02, 1.71591332e-02, 8.38915649e-03],
       [7.91290549e-03, 2.30600993e-10, 9.05891217e-03, 2.44365759e-03]])

In [ ]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [ ]:
print ('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

Item-based CF MSE: 18.550991865684374


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(traind) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pm=pd.DataFrame(cosine_similarity(r_mat_tr))
#pm.head()
ctrain = cosine_similarity(r_mat_tr)


[0.0444643  0.02349345 0.01776265 0.01757378 0.01775221 0.01686932
 0.01620957 0.01499802 0.01488605 0.01141899 0.0112713  0.01116964
 0.0108935  0.01031012 0.01017654 0.01009798 0.00951854 0.00963973
 0.0091539  0.00933332 0.00930415 0.00896018 0.00833648 0.00842297
 0.00832996 0.00820888 0.00820571 0.00793285 0.00780983 0.00737966
 0.00733475 0.00690053 0.00707393 0.00687504 0.00678373 0.00657981
 0.00651143 0.00650772 0.00648156 0.00632685 0.0062967  0.00589503
 0.00587839 0.00547108 0.00553978 0.00548993 0.00546542 0.00533141
 0.00525879 0.00513267]
0.5030181333838422


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(testd) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pmtt=pd.DataFrame(cosine_similarity(r_mat_tr))
#print (pmtt[:2])
#pmtt.head()
ctest = cosine_similarity(r_mat_tr)

[0.05382399 0.04783327 0.04837764 0.03635407 0.02647748 0.02225534
 0.01999909 0.02050899 0.01973338 0.01882333 0.01881439 0.01668118
 0.01645692 0.01650472 0.01607021 0.01604763 0.01566951 0.01369147
 0.01298814 0.01264341 0.01261496 0.01215978 0.011942   0.01146232
 0.01154127 0.01115682 0.01039123 0.01051502 0.01070081 0.01030295
 0.01030903 0.00991172 0.00976845 0.0094579  0.00964448 0.00953174
 0.00915143 0.00905646 0.0088877  0.00854731 0.00862895 0.00831478
 0.00849652 0.00841647 0.00830525 0.00823068 0.00819722 0.00806327
 0.00794774 0.00792961]
0.7593380170113238


In [ ]:
print (' CF MSE: ' + str(get_mse(ctrain, ctest)))

 CF MSE: 0.04280465084217405


In [ ]:

df1 = df1.sort_values(by='overall')
df1 = df1.reset_index(drop=True)
count_users = df1.groupby("reviewerID", as_index=False).count()

In [ ]:
count = df1.groupby("asin", as_index=False).mean()

In [ ]:
items_df = count[['asin']]
items_df.head()
print(len(items_df))

290


In [ ]:
users_df = count_users[['reviewerID']]
users_df.head()
print(len(users_df))

17234


In [ ]:

users_list = users_df.values
len(users_list)

17234

In [ ]:
df_clean_matrix = df1.pivot(index='asin', columns='reviewerID', values='overall').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).to_numpy()
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.03448276, -0.03448276, -0.03448276, ..., -0.03448276,
        -0.03448276, -0.03448276],
       [-0.01034483, -0.01034483, -0.01034483, ..., -0.01034483,
        -0.01034483, -0.01034483],
       [-0.03448276, -0.03448276, -0.03448276, ..., -0.03448276,
        -0.03448276, -0.03448276],
       ...,
       [-0.03103448, -0.03103448, -0.03103448, ..., -0.03103448,
        -0.03103448, -0.03103448],
       [-0.01724138, -0.01724138, -0.01724138, ..., -0.01724138,
        -0.01724138, -0.01724138],
       [-0.03448276, -0.03448276, -0.03448276, ..., -0.03448276,
        -0.03448276, -0.03448276]])

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned)

In [ ]:
sigma = np.diag(sigma)


In [ ]:

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_clean_matrix.columns)
preds_df['userID'] = users_df
preds_df.set_index('userID', inplace=True)
preds_df.head()

asin,B0009B0IX4,B000GAO9T2,B000I8ACMU,B000IEB00W,B0013G8PTS,B001CJTE0K,B001CL2CKW,B001DETCUG,B001FVPU9O,B001GD87QO,B001XXUOQI,B0027FFZEW,B002D4IHYM,B002M3TLLC,B0030C4K8I,B003QSYK2C,B003RDV0VU,B003RZTCWC,B003SHM08W,B003SX0P1A,B003TM5AJM,B003UC8RVE,B003UC93WQ,B003X5OBYU,B00400VOJ2,B0042TY68C,B0047EM82I,B004877KOK,B004AJ8PG8,B004CU1VW0,B004FSKVIE,B004G1L52Q,B004I58ZVY,B004I6F3E0,B004I8WD6E,B004I8WD7S,B004IAJIHE,B004LJ8N78,B004LSN0GS,B004M5KV3A,...,B00DQCUVPI,B00DRJTY44,B00DWKK6PE,B00E24KTYM,B00E2TT8W6,B00E2YI052,B00E327OIM,B00E45043A,B00E486Q9I,B00EB1BY36,B00EDB938C,B00EF1OGOG,B00ENOAYVY,B00EPLTC1I,B00ESJGILE,B00EV8Y50S,B00F3SISYO,B00F3YSN5M,B00F4A8FQM,B00F8OYPUO,B00F8P9UFS,B00FH669I4,B00FH66AW4,B00FH6QTJ8,B00FJJ38HY,B00FK0ECYA,B00FME4NAW,B00FQPGAHG,B00FZ14E3S,B00GBFQWEG,B00GG5IHJY,B00GH0JXAK,B00GWR36F6,B00GYD782A,B00H2RP6W6,B00I3ULLNO,B00I3ULVLQ,B00I3UWD40,B00I3UWD7C,B00KB9AWEE
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A01038432MVI9JXYTTK5T,0.031854,0.003863,0.007371,0.013102,0.037893,0.013138,0.016339,0.014480,0.033188,0.004608,0.003897,0.034018,0.012675,0.011763,0.064593,0.023531,0.022153,0.040938,0.022411,0.010089,0.062937,0.051884,0.048939,0.054329,0.029818,0.032491,0.015812,0.010824,0.004584,0.015893,0.033682,-0.003689,0.003232,0.013459,0.018987,0.023802,0.017161,0.012465,0.051678,0.207300,...,0.006478,0.018677,0.011102,0.005690,0.019916,0.023363,0.026218,0.001093,0.032898,0.008531,0.057782,0.018508,0.014731,0.006145,0.022553,0.003744,0.048834,0.040747,0.004313,0.009044,0.020364,0.022372,0.016601,0.016105,0.001373,0.011728,0.014315,0.004180,0.003368,0.001313,0.021290,0.016370,0.011687,0.012833,0.012138,0.004313,0.004277,0.012557,0.003936,0.000094
A01335502AS0LP3ASV1Z7,0.010564,0.010749,0.011162,0.010795,0.029655,0.011490,0.010931,0.011111,0.010798,0.008683,0.014475,0.009390,0.012930,0.008131,0.011333,0.011739,0.011273,0.011709,0.011906,0.011006,0.013903,0.010993,0.012492,0.012325,0.012010,0.011429,0.011659,0.010628,0.010406,0.011399,0.013188,0.014940,0.011675,0.011567,0.009318,0.009206,0.011486,0.011109,0.012974,0.010826,...,0.011307,0.009574,0.009710,0.011449,0.008101,0.009870,0.007959,0.011400,0.009033,0.011320,0.012242,0.011714,0.011803,0.011108,0.001939,0.011379,0.009168,0.012574,0.010922,0.003866,0.011775,0.009545,0.007835,0.008851,0.012000,0.009539,0.015666,0.011120,0.011923,0.009544,0.012026,0.012185,0.011904,0.011279,0.009898,0.011127,0.011383,0.011414,0.011234,0.012632
A01410041ZE038XXVXLBB,0.007715,-0.006255,-0.012576,-0.006274,-0.035302,-0.004423,-0.000724,-0.005869,0.030840,-0.001848,-0.038231,0.019570,-0.006849,0.013307,0.049173,0.000801,0.005221,0.025106,0.006906,0.015862,0.023203,0.048136,0.065107,0.035571,0.011489,0.008322,-0.005575,-0.003682,0.033320,-0.001041,0.023027,-0.002284,-0.010214,-0.008856,0.011621,0.020836,0.013859,-0.001115,0.031716,0.278857,...,-0.016466,0.050899,-0.000307,-0.016327,0.010712,0.006856,0.020539,-0.011364,0.048081,-0.008218,0.043479,-0.006482,-0.015135,-0.014152,0.057989,-0.017623,0.085957,0.010880,-0.018879,0.031135,-0.006870,0.013411,0.030719,0.022857,-0.028932,0.004271,-0.022602,-0.016191,-0.021488,-0.004969,-0.004592,-0.005685,-0.020934,-0.010484,0.001531,-0.014764,-0.012873,-0.010604,-0.015721,-0.020667
A01534622D57NTXSH2WBR,0.046009,0.049383,0.051768,0.050105,0.191139,0.053958,0.049673,0.050678,0.050734,0.033530,0.074794,0.037967,0.066266,0.030539,0.053027,0.055307,0.053430,0.055842,0.057526,0.054102,0.068936,0.051981,0.066691,0.060217,0.057866,0.052789,0.054806,0.047768,0.052436,0.053757,0.067651,0.084515,0.056174,0.054161,0.039110,0.038534,0.056094,0.051729,0.065008,0.061732,...,0.051983,0.047106,0.040942,0.053173,0.028808,0.041194,0.028086,0.054174,0.040291,0.053072,0.060470,0.054999,0.054998,0.050755,-0.012891,0.052626,0.044127,0.060496,0.050502,-0.000038,0.054891,0.040048,0.030112,0.037007,0.056152,0.040271,0.084060,0.051200,0.056287,0.040255,0.056945,0.058875,0.055346,0.052143,0.042679,0.051361,0.053482,0.052907,0.051848,0.0

In [ ]:
preds_df.shape

(17234, 290)

In [ ]:
def recommend_it(predictions_df, itm_df, original_ratings_df, num_recommendations=10,rreviewerID='A01038432MVI9JXYTTK5T'):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[rreviewerID].sort_values(ascending=False)
    
    # Get the user's data and merge in the item information.
    user_data = original_ratings_df[original_ratings_df.reviewerID == rreviewerID]
    user_full = (user_data.merge(itm_df, how = 'left', left_on = 'asin', right_on = 'asin').
                     sort_values(['overall'], ascending=False)
                 )

    print ('User {0} has already purchased {1} items.'.format(rreviewerID, user_full.shape[0]))
    print ('Recommending the highest {0} predicted  items not already purchased.'.format(num_recommendations))
    
    # Recommend the highest predicted rating items that the user hasn't bought yet.
    recommendations = (itm_df[~itm_df['asin'].isin(user_full['asin'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'asin',
               right_on = 'asin').
         rename(columns = {rreviewerID: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    topk=recommendations.merge(original_ratings_df,left_index = True, right_on = 'asin',left_on='asin').drop_duplicates(
    ['asin', 'title'])[['asin', 'title']]

    return topk

In [ ]:
recommend_it(preds_df, items_df, df1, 5)

User A01038432MVI9JXYTTK5T has already purchased 2 items.
Recommending the highest 5 predicted  items not already purchased.


,asin,title
1046,B0059DLLJC,Snap-on Rubber Coated Case for Apple&reg; iPho...
1541,B004M5KV3A,OtterBox Universal Defender Case for iPhone 4 ...
532,B005JHIYLG,Aimo Wireless Ip-case Premium Chrome Aluminum ...
542,B005LUBUT4,Carrying Case for iPhone 4 - Non-Retail Packag...
154,B0030C4K8I,Motorola H720 Black Bluetooth Headset - Retail...


In [ ]:
#recommend for any user
recommend_it(preds_df, items_df, df1, 5,'A01038432MVI9JXYTTK5T')

User A01038432MVI9JXYTTK5T has already purchased 2 items.
Recommending the highest 5 predicted  items not already purchased.


,asin,title
1046,B0059DLLJC,Snap-on Rubber Coated Case for Apple&reg; iPho...
1541,B004M5KV3A,OtterBox Universal Defender Case for iPhone 4 ...
532,B005JHIYLG,Aimo Wireless Ip-case Premium Chrome Aluminum ...
542,B005LUBUT4,Carrying Case for iPhone 4 - Non-Retail Packag...
154,B0030C4K8I,Motorola H720 Black Bluetooth Headset - Retail...
